# ML E2022 - Hand in 3

This project is about using a hidden Markov model for gene finding in prokaryotes as explained in class and explored in the exercises in week 47.

You must do the project in groups similar to the previous hand ins. The project is mandatory and each group must hand in a report describing their work and results as outlined in section Report below. Handin the report in pdf-format via BrightSpace no later than **Friday, December 9, 2022, at 23:59**.

### Data set

You are give a data set containing 10
[Staphylococcus](http://en.wikipedia.org/wiki/Staphylococcus) genomes,
each containing several genes (i.e. substring) obeying the "gene syntax" explained in class.
The genomes are between 1.8 million and 2.8 million nucleotides long.
For 5 of the genomes, you are also given the gene structure, i.e. the location of the genes.
For the remaining 5 genomes, you only know that they contain genes according to the "gene syntax". You have already explored two of the genomes in the practical exercises in week 47.

The genomes and their annontations are given in
[FASTA format](http://en.wikipedia.org/wiki/Fasta_format).
There are several Python libraries for reading sequences in FASTA format (use Google to find one),
and you are welcome to use the simple reader function `read_fasta_file` below that you also used in the exercises in week 47.

The data is:


  * The files [genome1.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome1.fa),
    [genome2.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome2.fa),
    [genome3.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome3.fa),
    [genome4.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome4.fa),
    [genome5.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome5.fa),
    [genome6.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome6.fa),
    [genome7.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome7.fa),
    [genome8.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome8.fa),
    [genome9.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome9.fa), and
    [genome10.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome10.fa) contain the 10 genomes.


* The files [true-ann1.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/true-ann1.fa),
    [true-ann2.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/true-ann2.fa),
    [true-ann3.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/true-ann3.fa),
    [true-ann4.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/true-ann4.fa), and
    [true-ann5.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/true-ann5.fa) contain the annotation
    of the first 5 genomes. The annotation is given in FASTA format as a sequence
    over the symbols `N`, `C`, and `R`. The symbol `N`, `C`, or `R` at position _i_ in
    true-ann_k_.fa gives the "state" of the nucleotide at position _i_ in
    genome_k_.fa. `N` means that the nucleotide is non-coding. `C` means that the
    nucleotide is coding and part of a gene in the direction from left to right. `R`
    means that the nucleotide is coding and part of gene in the reverse direction
    from right to left. This is exactly as in the exercises in week 47.  


* All the above files are available in [data-handin3.zip](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/data-handin3.zip)

A simple analysis of the annotations and the corresponding genomes, it is
clear that fx several start-codons are possible (and not only atg as modelled
in class).


In [ ]:
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

The `read_fast_file` function is used as follows. First, we write an example FASTA file:

In [3]:
s = '''
;Example FASTA file
>genomeA
CGATTAAAGA
TAGAAATACA
>annotationA
CCCCCCNNNN
NNNNRRRRRR
'''
with open('test.fa', 'w') as fp:
    fp.write(s)

When we read the simple FASTA file `test.fa` using `read_fasta_file`, we get a dictionary with the keys `genomeA` and `annotationA` corresponding to the contents of the above file.

In [4]:
x = read_fasta_file('test.fa')
x

{'genomeA': 'CGATTAAAGATAGAAATACA', 'annotationA': 'CCCCCCNNNNNNNNRRRRRR'}

In [5]:
print(x['genomeA'])

CGATTAAAGATAGAAATACA


### Problem

Your task is to predict the gene structure of the 5 unannotated genomes, i.e.
[genome6.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome6.fa),
[genome7.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome7.fa),
[genome8.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome8.fa),
[genome9.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome9.fa), and
[genome10.fa](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/genome10.fa), in the best possible manner using an HMM based approach.

As explained in class, predicting gene structure using an HMM involves:

  * Deciding on an initial model structure, i.e. the number of hidden states
    and which transitions and emission should have a fixed probability (e.g. 0
    for "not possible", or 1 for "always the case").
    You might get inspiration from, or even use, one of the model structures discussed in class.
    However, be aware that the models presented in class model only one start-codon atg. 
    This is not the case in the presented data, where multiple start-codons are possible (as also mentioned in the     lecture/slides about this project).


  * Tune model parameters by training, i.e. set the non-fixed emission and transition probabilities.
    This can be done by several methods as explained in class. You can use
    (1) Training-by-Counting using the 5 genomes with know gene structure because you
    know the underlying hidden state for each observation (i.e. symbol) in these
    sequences, or
    (2) Viterbi- or EM-training using all the 10 genomes,
    including the 5 genomes with unknown gene structure.
    You can of course also combine the two approaches.
    You can evaluate the performance of your gene finder by computing the
    approximate correlation (AC) between your predictions and
    the true structure using this small python program
    [compare_anns.py](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/compare_anns.py)
    (see [this paper](http://www.sciencedirect.com/science/article/pii/S0888754396902980)
    for details).

    In this project, Training-by-Counting is mandatory, while Viterbi- or
    EM-training is optional. If you implement EM-training you should of course be
    aware of the numerical problems of the forward- and backward-algorithms unless you use scaling or similar techniqes.

* Evaluate the performance of your gene predictor. In this project, you must do a 5-fold cross validation on the 5 genomes with known gene structure, i.e.:

    You consider genome 1 to 5 in five rounds. In a each you train your model using Training-by-Counting on the remaining 4 genomes, predict the gene structure of the genome you consider, and compute the approximate correlation coefficient (AC) between your predicted gene structure and the true gene structure using the python program  [compare_anns.py](https://birc.au.dk/~cstorm/courses/ML_e22/projects/handin3/compare_anns.py).


  * Use your best model to predict the gene structure for the 5 unannotated
    genomes using the Viterbi algorithm with subsequent backtracking.
    I.e. for each unannotated genome you must find a most likely sequence of
    states in your model generating it, and convert this sequence of states into a
    FASTA file giving the annotation of each nucleotide as `N`, `C`, or `R`.
    You should make files, `pred-ann6.fa`, ..., `pred-ann10.fa`, giving the
    annotation for the 5 unannotated genomes. 
    
    You can use the www-service [GeneFinder Verifier](https://services.birc.au.dk/genefinder-verifier/) to compare your predictions on Genome 6-10 against their true structures.
   

### Report

You (i.e. your group) must hand in a short report (in pdf-format) describing your
work and results and a zip-archive of the files `pred-ann6.fa` and
`pred-ann7.fa`,..., `pred-ann10.fa` containing your predictions on the 5
genomes with unknown structure. The files _must_ be in FASTA format similar to
the annotation of the other genomes. Handin the report and zip-archive via
Blackboard no later than **Friday, December 9, 2022, at 23:59**. Your report should
be no more than 3 pages. It must cover:

  * The status of the work, i.e. does it work, if not, then why.


  * An explanation and illustration of your model structure (i.e. a transition
    diagram). This should include an explanation of how you model start and stop codons, and an explanation of how you have trained your model. For Training-by-Counting you might comment on how you 
    translata the given annotations of `C`, `N`, and `R`'s into corresponding sequences
    of hidden states. The transition-diagram must contain all non-0 emission- and transition probabilities
    for the model that you have used for predicting the gene structure of the 5 genomes with unknown gene structure.


  * An explanation of how you have predicted the gene structure for the the 5 genomes with unknown gene structure. You should comment on how you translate a most likely sequence of hidden states as returned by Viterbi decoding into a sequence of `C`, `N`, and `R`'s.


  * The result of your 5-fold cross valiation on the 5 genomes with known gene structure. The report must include a table with the AC obtained in each of the five rounds of the 5-fold cross validation.
    
    
  * The result of comparing your predictions on the 5 genomes with unknown gene structure against their true structures via the www-service [GeneFinder Verifer](https://services.birc.au.dk/genefinder-verifier/).